In [1]:
import pandas as pd
import re
import requests
import xmltodict
from fuzzywuzzy import process

In [2]:
path = "data/rollcallvote.pdf"

In [3]:
from PyPDF2 import PdfReader


# Ouvrir le fichier PDF
with open(path, 'rb') as f:
    # Create a PyPDF2 reader object
    pdf = PdfReader(f)
    
    # Extract the text from the PDF
    text = ''
    for page in pdf.pages:
        text += page.extract_text()

In [6]:
text

"P9_PV(2020)10-06(RCV)_FR.docx21PE 659.2027. A9-0170/2020 - Michal Šimečka - § 506/10/2020 12:40:24.939535+ECR: Bourgeois, Ďuriš Nicholsonová, Jurzyca, Kanko, Van OvertveldtGUE/NGL: Arvanitis, Aubry, Barrena Arza, Björk, Bompard, Chaibi, Daly, Demirel, Ernst, Georgoulis, Gusmão, Hazekamp, Kizilyürek, Kokkalis, Kouloglou, Kountoura, MacManus, Matias, Maurel, Michels, Modig, Omarjee, Papadimoulis, Pelletier, Pineda, Rego, Rodríguez Palop, Schirdewan, Scholz, Urbán Crespo, Villanueva Ruiz, Villumsen, WallaceID: KofodNI: Adinolfi Isabella, Beghin, Buschmann, Castaldo, Comín i Oliveres, Corrao, D'Amato, Evi, Ferrara, Furore, Gemma, Giarrusso, Gyöngyösi, Konstantinou, Pedicini, Pignedoli, Ponsatí Obiols, Puigdemont i Casamajó, Rondinelli, Rookmaker, Sonneborn, ZulloPPE: Adamowicz, Ademov, Alexandrov Yordanov, Amaro, Arias Echeverría, Arimont, Arłukowicz, Asimakopoulou, Băsescu, Bellamy, Benjumea Benjumea, Bentele, Berendsen, Berger, Bernhuber, Bilčík, Blaga, Bogdan, Bogovič, Buda, Buşoi, Buz

In [4]:
footer_regex = ".docx\d{2}PE\s\d{1,3}\.\d{1,3}"
text_without_footer = re.split(footer_regex, text)[1]
filter_prefix_regex = r"\d{2}:[\d.:]+"
test_text = re.split(filter_prefix_regex, text_without_footer)[1]
pattern_pour = r"\+([^\d]+)"
matches_pour = re.findall(pattern_pour, test_text)[0]
contre_text = re.split(matches_pour, text)[1]
contre_text_without_footer = re.split(footer_regex, contre_text)[1]
pattern_contre = r"([^\d]+)"
matches_contre = re.findall(pattern_contre, contre_text_without_footer)[0]
abstention_text = re.split(matches_contre, text)[1]
footer_regex = ".docx\d{2}PE\s\d{1,3}\.\d{1,3}"
abstention_text_without_footer = re.split(footer_regex, abstention_text)[0]

In [5]:
partis_politiques = ["ECR", "GUE/NGL", "ID", "NI", "PPE", "Renew", "S&D", "Verts/ALE"]
ecr_exclusions = r"(?:GUE/NGL|ID|NI|PPE|Renew|S\&D|Verts/ALE)"
gnu_ngl_exclusions = r"(?:ID|NI|PPE|Renew|S\&D|Verts/ALE)"
id_exclusions = r"(?:NI|PPE|Renew|S\&D|Verts/ALE)"
ni_exclusions = r"(?:PPE|Renew|S\&D|Verts/ALE)"
ppe_exclusions = r"(?:Renew|S\&D|Verts/ALE)"
renew_exclusions = r"(?:S\&D|Verts/ALE)"
sd_exclusions = r"(?:Verts/ALE)"
votes_by_group = {
    "+":{},
    "-":{},
    "0":{},
}
final_results = pd.DataFrame({"fullName":[], "vote":[], "group":[]})

In [503]:
matches_pour_ECR = re.split(ecr_exclusions, matches_pour)[0]
ECR_POUR = re.sub( "ECR:", "",matches_pour_ECR).lstrip().split(",")
final_results = format_name_and_vote("ECR", ECR_POUR, "+", final_results)

gnu_ngl_text = re.split(matches_pour_ECR, matches_pour)[-1]
matches_pour_GNUNGL = re.split(gnu_ngl_exclusions, gnu_ngl_text)[0]
GNUNGL_POUR = re.sub( "GUE/NGL:", "",matches_pour_GNUNGL).lstrip().split(",")
final_results = format_name_and_vote("GUE/NGL", GNUNGL_POUR, "+", final_results)

id_text = re.split(matches_pour_GNUNGL, matches_pour)[-1]
matches_pour_ID = re.split(id_exclusions, id_text)[0]
ID_POUR = re.sub( "ID:", "",matches_pour_ID).lstrip().split(",")
final_results = format_name_and_vote("ID", ID_POUR, "+", final_results)

ni_text = re.split(matches_pour_ID, matches_pour)[-1]
matches_pour_NI = re.split(ni_exclusions, ni_text)[0]
NI_POUR = re.sub( "NI:", "",matches_pour_NI).lstrip().split(",")
final_results = format_name_and_vote("NI", NI_POUR, "+", final_results)

ppe_text = re.split(matches_pour_NI, matches_pour)[-1]
matches_pour_PPE = re.split(ppe_exclusions, ppe_text)[0]
PPE_POUR = re.sub( "PPE:", "",matches_pour_PPE).lstrip().split(",")
final_results = format_name_and_vote("PPE", PPE_POUR, "+", final_results)

renew_text = re.split(matches_pour_PPE, matches_pour)[-1]
matches_pour_Renew = re.split(renew_exclusions, renew_text)[0]
Renew_POUR = re.sub( "Renew:", "",matches_pour_Renew).lstrip().split(",")
final_results = format_name_and_vote("Renew", Renew_POUR, "+", final_results)

sd_text = re.split(matches_pour_Renew, matches_pour)[-1]
matches_pour_sd = re.split(sd_exclusions, sd_text)[0]
SD_POUR = re.sub( "S&D:", "",matches_pour_sd).lstrip().split(",")
final_results = format_name_and_vote("S&D", SD_POUR, "+", final_results)

VertsALE_text = re.split(matches_pour_sd, matches_pour)[-1]
VertsALE_POUR = re.sub( "Verts/ALE:", "",VertsALE_text).lstrip().split(",")
final_results = format_name_and_vote("Verts/ALE", VertsALE_POUR, "+", final_results)


In [504]:
matches_contre_ECR = re.split(ecr_exclusions, matches_contre)[0]
ECR_CONTRE = re.sub( "ECR:", "",matches_contre_ECR.lstrip()).split(",")
final_results = format_name_and_vote("ECR", ECR_CONTRE, "-", final_results)

gnu_ngl_text_contre = re.split(matches_contre_ECR, matches_contre)[-1]
matches_contre_GNUNGL = re.split(gnu_ngl_exclusions, gnu_ngl_text_contre)[0]
GNUNGL_CONTRE = re.sub( "GUE/NGL:", "",matches_contre_GNUNGL).lstrip().split(",")
final_results = format_name_and_vote("GUE/NGL", GNUNGL_CONTRE, "-", final_results)


id_text_contre = re.split(matches_contre_GNUNGL, matches_contre)[-1]
matches_contre_ID = re.split(id_exclusions, id_text_contre)[0]
ID_CONTRE = re.sub( "ID:", "",matches_contre_ID).lstrip().split(",")
final_results = format_name_and_vote("ID", ID_CONTRE, "-", final_results)

ni_text_contre = re.split(matches_contre_ID, matches_contre)[-1]
matches_contre_NI = re.split(ni_exclusions, ni_text_contre)[0]
NI_CONTRE = re.sub( "NI:", "",matches_contre_NI).lstrip().split(",")
final_results = format_name_and_vote("NI", NI_CONTRE, "-", final_results)

ppe_text_contre = re.split(matches_contre_NI, matches_contre)[-1]
matches_contre_PPE = re.split(ppe_exclusions, ppe_text_contre)[0]
PPE_CONTRE = re.sub( "PPE:", "",matches_contre_PPE).lstrip().split(",")
final_results = format_name_and_vote("PPE", PPE_CONTRE, "-", final_results)

renew_text_contre = re.split(matches_contre_PPE, matches_contre)[-1]
matches_contre_Renew = re.split(renew_exclusions, renew_text_contre)[0]
Renew_CONTRE = re.sub( "Renew:", "",matches_contre_Renew).lstrip().split(",")
final_results = format_name_and_vote("Renew", Renew_CONTRE, "-", final_results)

sd_text_contre = re.split(matches_contre_Renew, matches_contre)[-1]
matches_contre_sd = re.split(sd_exclusions, sd_text_contre)[0]
SD_CONTRE = re.sub( "S&D:", "",matches_contre_sd).lstrip().split(",")
final_results = format_name_and_vote("S&D", SD_CONTRE, "-", final_results)


VertsALE_text_contre = re.split(matches_contre_sd, matches_contre)[-1]
VertsALE_CONTRE = re.sub( "Verts/ALE:", "",VertsALE_text_contre).lstrip().split(",")
final_results = format_name_and_vote("Verts/ALE", VertsALE_CONTRE, "-", final_results)


In [505]:
matches_abstention_ECR = re.split(ecr_exclusions, abstention_text_without_footer)[0]
ECR_ABS = re.sub( "ECR:", "",matches_contre_ECR).lstrip().split(",")
final_results = format_name_and_vote("ECR", ECR_ABS, "0", final_results)

gnu_ngl_text_abs = re.split(matches_abstention_ECR, abstention_text_without_footer)[-1]
matches_abs_GNUNGL = re.split(gnu_ngl_exclusions, gnu_ngl_text_abs)[0]
GNUNGL_ABS = re.sub("GUE/NGL:", "",matches_abs_GNUNGL).lstrip().split(",")
final_results = format_name_and_vote("GUE/NGL", GNUNGL_ABS, "0", final_results)

id_text_abs = re.split(matches_abs_GNUNGL, abstention_text_without_footer)[-1]
matches_abs_ID = re.split(id_exclusions, id_text_abs)[0]
ID_ABS = re.sub("ID:", "",matches_abs_ID).lstrip().split(",")
final_results = format_name_and_vote("ID", ID_ABS, "0", final_results)

ni_text_abs = re.split(matches_abs_ID, abstention_text_without_footer)[-1]
matches_abs_NI = re.split(ni_exclusions, ni_text_abs)[0]
NI_ABS = re.sub("NI:", "",matches_abs_NI).lstrip().split(",")
final_results = format_name_and_vote("NI", NI_ABS, "0", final_results)

ppe_text_abs = re.split(matches_abs_NI, abstention_text_without_footer)[-1]
matches_abs_PPE = re.split(ppe_exclusions, ppe_text_abs)[0]
PPE_ABS = re.sub("PPE:", "",matches_abs_PPE).lstrip().split(",")
final_results = format_name_and_vote("PPE", PPE_ABS, "0", final_results)

renew_text_abs = re.split(matches_abs_PPE, abstention_text_without_footer)[-1]
matches_abs_Renew = re.split(renew_exclusions, renew_text_abs)[0]
Renew_ABS = re.sub("Renew:", "",matches_abs_Renew).lstrip().split(",")
final_results = format_name_and_vote("Renew", Renew_ABS, "0", final_results)

sd_text_abs = re.split(matches_abs_Renew, abstention_text_without_footer)[-1]
matches_abs_sd = re.split(sd_exclusions, sd_text_abs)[0]
SD_ABS = re.sub("S&D:", "",matches_abs_sd).lstrip().split(",")
final_results = format_name_and_vote("S&D", SD_ABS, "0", final_results)

VertsALE_text_abs = re.split(matches_abs_sd, abstention_text_without_footer)[-1]
VertsALE_ABS = re.sub("Verts/ALE:", "",VertsALE_text_contre).lstrip().split(",")
final_results = format_name_and_vote("Verts/ALE", VertsALE_ABS, "0", final_results)

In [506]:
final_results

,fullName,vote,group
0,Geert BOURGEOIS,+,ECR
1,Lucia ĎURIŠ NICHOLSONOVÁ,+,ECR
2,Eugen JURZYCA,+,ECR
3,Assita KANKO,+,ECR
4,Johan VAN OVERTVELDT,+,ECR
...,...,...,...
739,Mislav KOLAKUŠIĆ,0,NI
740,Ivan Vilibor SINČIĆ,0,NI
741,Riho TERRAS,0,PPE
742,Asim ADEMOV,0,Renew


In [267]:
columns = ECR_POUR + GNUNGL_POUR + ID_POUR + NI_POUR + PPE_POUR + Renew_POUR + SD_POUR + VertsALE_POUR + ECR_CONTRE + GNUNGL_CONTRE + ID_CONTRE + NI_CONTRE + PPE_CONTRE + Renew_CONTRE + SD_CONTRE + VertsALE_CONTRE + ECR_ABS + GNUNGL_ABS + ID_ABS + NI_ABS + PPE_ABS + Renew_ABS + SD_ABS + VertsALE_ABS

In [269]:
len(set(columns))

694

In [434]:
url = "https://www.europarl.europa.eu/meps/en/full-list/xml"

def get_meps():
    response = requests.get(url)
    data = xmltodict.parse(response.content)
    return data["meps"]["mep"]

In [435]:
meps = get_meps()

In [436]:
meps_df = pd.DataFrame.from_dict(meps, orient="columns")

In [493]:
political_groups_matching = {
    "Group of the European People's Party (Christian Democrats)":"PPE",
    "Identity and Democracy Group":"ID",
    "Group of the Progressive Alliance of Socialists and Democrats in the European Parliament":"S&D",
    "European Conservatives and Reformists Group":"ECR",
    "Group of the Greens/European Free Alliance":"Verts/ALE",
    "Renew Europe Group":"Renew",
    "The Left group in the European Parliament - GUE/NGL":"GUE/NGL",
    "Non-attached Members":"NI",
}

In [494]:
# meps_2 = meps_df.assign(shortPoliticalGroup=lambda x: political_groups_matching[x.politicalGroup])
meps_df["shortPoliticalGroup"] = meps_df.apply(lambda x: political_groups_matching[x.politicalGroup], axis=1)

In [501]:
def format_name_and_vote(group, group_votes, vote, results_df):
    not_found = []
    result = {
        'fullName':[],
        'group':[],
        'vote':[]
    }
    for mep in group_votes:
        if mep != '':
            fullName = process.extract(mep, meps_df["fullName"][meps_df["shortPoliticalGroup"] == group].to_list(), limit=1)[0]
            if fullName[1] > 85:
                result["fullName"].append(fullName[0]),
                result["group"].append(group),
                result["vote"].append(vote)
            else:
                fullName = process.extract(mep, meps_df["fullName"].to_list(), limit=1)[0]
                not_found.append(fullName[0])
                result["fullName"].append(fullName[0]),
                result["group"].append(group),
                result["vote"].append(vote)
    groups_df = pd.DataFrame(result)
    return pd.concat([results_df, groups_df], ignore_index=True)

In [357]:
format_name_and_vote(group, group_votes, vote)

('Geert BOURGEOIS', 90)
('Veronika VRECIONOVÁ', 46)
('Eugen JURZYCA', 90)
('Assita KANKO', 90)
('Johan VAN OVERTVELDT', 95)


[{'fullName': 'Geert BOURGEOIS', 'group': 'ECR', 'vote': '+'},
 {'fullName': 'Lucia ĎURIŠ NICHOLSONOVÁ', 'group': 'ECR', 'vote': '+'},
 {'fullName': 'Eugen JURZYCA', 'group': 'ECR', 'vote': '+'},
 {'fullName': 'Assita KANKO', 'group': 'ECR', 'vote': '+'},
 {'fullName': 'Johan VAN OVERTVELDT', 'group': 'ECR', 'vote': '+'}]

In [485]:
meps_df["shortPoliticalGroup"][meps_df["shortPoliticalGroup"]==('Adinolfi Isabella', 90)[0]].values[0]

4      SD
6      SD
8      SD
14     SD
19     SD
       ..
659    SD
684    SD
685    SD
688    SD
700    SD
Name: shortPoliticalGroup, Length: 143, dtype: object

In [486]:
meps_df[meps_df["shortPoliticalGroup"]=='SD']

,fullName,country,politicalGroup,id,nationalPoliticalGroup,shortPoliticalGroup
4,Alex AGIUS SALIBA,Malta,Group of the Progressive Alliance of Socialist...,197403,Partit Laburista,SD
6,Clara AGUILERA,Spain,Group of the Progressive Alliance of Socialist...,125045,Partido Socialista Obrero Español,SD
8,João ALBUQUERQUE,Portugal,Group of the Progressive Alliance of Socialist...,237224,Partido Socialista,SD
14,Andris AMERIKS,Latvia,Group of the Progressive Alliance of Socialist...,197783,Gods kalpot Rīgai,SD
19,Marc ANGEL,Luxembourg,Group of the Progressive Alliance of Socialist...,202073,Parti ouvrier socialiste luxembourgeois,SD
...,...,...,...,...,...,...
659,Petar VITANOV,Bulgaria,Group of the Progressive Alliance of Socialist...,197844,Bulgarian Socialist Party,SD
684,Tiemo WÖLKEN,Germany,Group of the Progressive Alliance of Socialist...,185619,Sozialdemokratische Partei Deutschlands,SD
685,Lara WOLTERS,Netherlands,Group of the Progressive Alliance of Socialist...,5392,Partij van de Arbeid,SD
688,Elena YONCHEVA,Bulgaria,Group of the Progressive Alliance of Socialist...,197842,Bulgarian Socialist Party,SD


In [489]:
mep="Agius Saliba"
group="SD"
print(process.extract(mep, meps_df["fullName"][meps_df["shortPoliticalGroup"] == group].to_list(), limit=1)[0])


('Alex AGIUS SALIBA', 95)
